In [1]:
import networkx as nx
import sys
import visJS2jupyter.visJS_module

print("Python version: " + sys.version)
print("networkx version: " + nx.__version__)

Python version: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
networkx version: 1.11


In [2]:
# The threshhold for filtering edges, based on the "combined_score" value assigned by STRING
EDGE_TOLERANCE = 0.5
# Name of the HTML file to output the network to
OUTPUT_FILE_NAME = "pretty_string.html"
# Rescaling factor for X-Y position of nodes
XY_SCALE = 600
# Rescaling factor for node sizes
NODE_SIZE_MULTIPLIER = 6

In [3]:
# Parse node information from string_network_coordinates.txt file
nodes = []
node_map = {}
node_info = []
num = 0

try:
    with open("string_network_coordinates.txt","r") as nodes_file:
        for line in nodes_file:
            line = line.split()
            if line[0][0] == "#":
                continue

            nodes.append(num)
            node_map[line[0]] = num
            node_info.append({"id":num,
                              "degree":1,
                              "node_label":line[0],
                              "x": float(line[1])*XY_SCALE,
                              "y": float(line[2])*XY_SCALE})
            num += 1
except IOError as e:
    print("Failed to open or parse 'string_network_coordinates.txt' to read in information about the nodes, exiting!")
    sys.exit(1)

In [4]:
# Parse edge information from string_interactions.tsv file
edges = []
edge_info = []

try:
    with open("string_interactions.tsv", "r") as edges_file:
        for line in edges_file:
            line = line.split()
            if line[0][0] == "#":
                continue

            edges.append( (node_map[line[0]], node_map[line[1]]) )
            edge_info.append({"source": node_map[line[0]],
                               "target": node_map[line[1]],
                               "color": "black",
                               "combined_score": float(line[-1]),
                               "title": "test"})
except IOError as e:
    print("Failed to open or parse 'string_interactions.tsv' to read in information about the edges, exiting!")
    sys.exit(1)

In [5]:
# Parse cluster information from string_kmeans_clusters.tsv file
try:
    with open("string_kmeans_clusters.tsv", "r") as clusters_file:
        for line in clusters_file:
            line = line.split()
            if line[0][0] == "#":
                continue

            node = node_map[line[4]]
            node_info[node]["cluster"] = line[1]
            node_info[node]["color"] = line[2]
except IOError as e:
    print("Could not open 'string_kmeans_clusters.tsv' to read in information about the clusters, will not include cluster information.")

In [6]:
# Generate the degree of a node based on the edges
# We actually compute the degree + 1, so that the disconnected
# nodes still show up on the graph
for edge in edge_info:
    source = edge["source"]
    target = edge["target"]
    
    node_info[source]["degree"] += 1
    node_info[target]["degree"] += 1

In [7]:
# Filter edges based on combined_score from STRING interaction network topology
for edge in edge_info:
    source = edge["source"]
    target = edge["target"]

    if edge["combined_score"] > EDGE_TOLERANCE:
        edge["color"] = "black"
    else:
        edge["color"] = "rgb(220,220,220)" # Grey

In [8]:
# Output network to HTML file, which may be opened in a web browser
output_html = visJS2jupyter.visJS_module.visjs_network(node_info, edge_info, 
                    output="html", node_label_field="node_label", node_size_multiplier=NODE_SIZE_MULTIPLIER)

with open(OUTPUT_FILE_NAME, "w") as output_file:
    output_file.write(output_html)